In [9]:
import numpy as np
import pandas as pd
np.set_printoptions(precision=3)

file = 'sensitivity_10_1e5.csv'

### Ratio misspecifications

In [10]:
sens = pd.DataFrame(np.loadtxt(f'../out/{file}'), columns=['r1', 'r2', 'r3', 't1', 't2', 't3', 'det', 'det_true'])
sens = sens.groupby(['r1', 'r2', 'r3', 't1', 't2', 't3']).agg(np.median).reset_index()
opt = sens[(sens.t1 == sens.r1) & (sens.t2 == sens.r2) & (sens.t3 == sens.r3)]\
            .drop(columns=['r1', 'r2', 'r3', 'det_true'])\
            .rename(columns={'det': 'det_opt'})

sens = sens.merge(opt, on=['t1', 't2', 't3'])\
           .assign(ratio=lambda x: x.det_true / x.det_opt)

sens

,r1,r2,r3,t1,t2,t3,det,det_true,det_opt,ratio
0,0.1,0.1,0.1,0.1,0.1,0.1,11.488274,11.488274,11.488274,1.000000
1,0.1,0.1,1.0,0.1,0.1,0.1,9.317828,11.481080,11.488274,0.999374
2,0.1,0.1,10.0,0.1,0.1,0.1,6.845921,11.351051,11.488274,0.988055
3,0.1,1.0,0.1,0.1,0.1,0.1,8.447628,11.448078,11.488274,0.996501
4,0.1,1.0,1.0,0.1,0.1,0.1,7.488980,11.449215,11.488274,0.996600
...,...,...,...,...,...,...,...,...,...,...
724,10.0,1.0,1.0,10.0,10.0,10.0,2.803985,1.971439,1.981430,0.994957
725,10.0,1.0,10.0,10.0,10.0,10.0,2.351205,1.971956,1.981430,0.995218
726,10.0,10.0,0.1,10.0,10.0,10.0,2.249044,1.981268,1.981430,0.999918
727,10.0,10.0,1.0,10.0,10.0,10.0,2.210031,1.980599,1.981430,0.999581


In [11]:
import plotly.graph_objects as go
from plotly.colors import DEFAULT_PLOTLY_COLORS
from plotly.subplots import make_subplots
import re

fig = make_subplots(3, 3, subplot_titles=[
    f'$\hat{{\eta}}_4 = {r3} \quad \eta = {eta}$' for r3 in [0.1, 1, 10] for eta in [0.1, 1, 10]
], horizontal_spacing=0.05, vertical_spacing=0.1)
for j, x in enumerate([0.1, 1, 10]):
    sens1 = sens[(sens.t1 == sens.t2) & (sens.t2 == sens.t3) & (sens.t1 == x)]
    for i, (r3, d1) in enumerate(sens1.groupby('r3')):
        z = d1.sort_values(['r2', 'r1']).ratio.to_numpy().reshape(3, 3)
        z = np.flipud(z)
        fig.add_trace(go.Heatmap(
            z=z*100,
            x=['0.1', '1', '10'],
            y=['10', '1', '0.1'],
            coloraxis='coloraxis',
            text=z*100,
            texttemplate='%{text:.2f}%',
            textfont={'size': 8}
        ), row=i+1, col=j+1)
fig.update_layout(
    coloraxis={'colorscale': 'viridis'},
    height=750,
)
fig.update_xaxes(title='$\hat{\eta}_2$', row=3)
fig.update_yaxes(title='$\hat{\eta}_3$', col=1)
fig.show()
fig.write_image('../figures/sensitivity.png')

In [12]:
import plotly.graph_objects as go
from plotly.colors import DEFAULT_PLOTLY_COLORS
from plotly.subplots import make_subplots
import re

fig = make_subplots(3, 3, subplot_titles=[
    f'$\eta_4 = {t3} \quad \hat{{\eta}} = {eta}$' for t3 in [0.1, 1, 10] for eta in [0.1, 1, 10]
], horizontal_spacing=0.05, vertical_spacing=0.1)

for j, x in enumerate([0.1, 1, 10]):
    sens1 = sens[(sens.r1 == sens.r2) & (sens.r2 == sens.r3) & (sens.r1 == x)]
    for i, (t3, d1) in enumerate(sens1.groupby('t3')):
        z = d1.sort_values(['t2', 't1']).ratio.to_numpy().reshape(3, 3)
        z = np.flipud(z)
        fig.add_trace(go.Heatmap(
            z=z*100,
            x=['0.1', '1', '10'],
            y=['10', '1', '0.1'],
            coloraxis='coloraxis',
            text=z*100,
            texttemplate='%{text:.2f}%',
            textfont={'size': 8}
        ), row=i+1, col=j+1)
fig.update_layout(
    coloraxis={'colorscale': 'viridis'},
    height=750,
)
fig.update_xaxes(title='$\eta_2$', row=3)
fig.update_yaxes(title='$\eta_3$', col=1)
fig.show()
fig.write_image('../figures/sensitivity2.png')

In [5]:
import plotly.express as px
from functools import reduce
from plotly.colors import DEFAULT_PLOTLY_COLORS

fig = go.Figure()
for i, r in enumerate([1, 2, 3]):
    cond = reduce(lambda x, y: x & y, [sens[f'r{rx}'] == sens.t1 for rx in [1, 2, 3] if rx != r])
    sens_temp = sens[(sens.t1 == sens.t2) & (sens.t2 == sens.t3) & cond]
    color = DEFAULT_PLOTLY_COLORS[i].split('(')[1].split(')')[0].split(', ')

    for j, eta in enumerate([0.1, 1, 10]):
        sens_eta = sens_temp[sens_temp.t1 == eta].sort_values(f'r{r}')
        fig.add_trace(go.Scatter(
            x=sens_eta[f'r{r}'], y=sens_eta.ratio, legendgroup=eta, legendgrouptitle_text=f'$\eta = {eta}$',
            name=f'$r_{r}$', line_color=f'rgba({", ".join(color)}, {(j+1)/3})'
        ))

fig.update_xaxes(type='log')
fig.update_layout(legend=dict(groupclick="toggleitem"))
fig.show()
# fig.write_image('r2.png')